In [6]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

load_dotenv()

openai.api_key  = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/payment.pdf")
pages = loader.load()

In [7]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("docs/_chat.txt",encoding='utf8')
pages = loader.load()

In [ ]:
page = pages[0]
print(page.page_content[0:100])

In [3]:
## split based on text
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150, 
    separators=["\n\n", "\n", " "]
)

In [10]:
docs = text_splitter.split_documents(pages)


In [11]:
len(docs)

2831

In [12]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [14]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [15]:
print(vectordb._collection.count())

2831


In [20]:
question = "S&P 500"

In [21]:
ans_docs = vectordb.similarity_search(question,k=4)

In [18]:
len(ans_docs)

4

In [22]:
ans_docs[0].page_content

"Could it be a technical glitch in the federal rebate policy?\n[11/29/23, 3:32:06\u202fPM] ~\u202fNilay Shah: R u going to own this system  or lease?\n[11/29/23, 3:49:51\u202fPM] ~\u202fKalpit: Own the system, I already have a system that is financed planning to add to it\n[11/29/23, 5:45:23\u202fPM] ~\u202fPrashant Jai: \u200eThis message was deleted.\n[11/29/23, 6:25:02\u202fPM] Nimish Pathak Austin: They recommend the S&P 500 for mainstream America as most people are not fluent in investing. It is a very safe bet with very good results as history shows. In reality, they bought individual companies after their expert analysis. And held it long term. Mostly value sector but now 50% portfolio is comprised of Apple. Both never accepted technology valuations as overbloated. Apple is the consumer/comm sector per them. \u200e<This message was edited>\n\u200e[11/29/23, 6:25:10\u202fPM] Nimish Pathak Austin: \u200eimage omitted\n[11/29/23, 6:30:16\u202fPM] ~\u202fVin Chikarmane: The S&P500 I

In [23]:
vectordb.persist()